# Data Chunking

#### This script converts the stock data into manageable data chunks and orders them on an yearly and quarterly bases in folders. The data can be saves as a csv file or alternatively as a pickle datafile

Load the necessary python libraries

In [2]:
import os
import pandas as pd

In [3]:
#!/usr/bin/env python
#title           :Stata_data_file_chunking.py
#description     :This will create a header for a python script.
#author          :Emmanuel Mensah Boateng
#date            :20240915
#version         :1.0
#usage           :python pyscript.py
#notes           :
#==============================================================================


# stata .dta file path
file_path = '../Data/dsws_variables_monthly_basic_2021.dta'

# Output folder for results 
output_dir = '../Data/dsws_Data/'  
os.makedirs(output_dir, exist_ok=True)  # Create directory if nonexistent

# data chunk size to process (e.g., 100,000 rows at a time)
chunk_size = 100000

# Function to extract the year and quarter from the data
#def extract_year_quarter(df, date_column):
#    df['year'] = pd.to_datetime(df['date']).dt.year
#    df['quarter'] = pd.to_datetime(df['date']).dt.to_period('Q')
    
    # Convert 'quarter' to a string to avoid issues
#    df['quarter'] = df['quarter'].astype(str)
    
    # Return the DataFrame with both columns at once to avoid fragmentation
#    return df

# Function to get the year and quarter from the data
def extract_year_quarter(df, date_column):
    # create copy
    df = df.copy()

    # add columns
    df['year'] = pd.to_datetime(df[date_column]).dt.year
    df['quarter'] = pd.to_datetime(df[date_column]).dt.to_period('Q').astype(str)
    print(df['quarter'])
    return df

# read .dta file chunk
chunk_iter = pd.read_stata(file_path, chunksize=chunk_size)

# rocess chunk
for i, chunk in enumerate(chunk_iter):
    print(f'Processing chunk {i+1}...')
    
    # Filter out only data where 'country' column has value 'usa'
    chunk = chunk.loc[chunk['country'] == 'usa']  # Use .loc[] to avoid SettingWithCopyWarning

    # skip this chunk if EOL
    if chunk.empty:
        continue
    
    # get year and quarter using date column
    chunk = extract_year_quarter(chunk, 'date') 

    # Group the chunk by year and quarter, and save it into separate pickle files
    for (year, quarter), group in chunk.groupby(['year', 'quarter']):
        # Create a subdirectory for each year
        year_dir = os.path.join(output_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)

        # define the filename for each quarter
        pickle_filename = os.path.join(year_dir, f'{year}_Q{quarter[-1]}.pkl')
        parquet_filename = os.path.join(year_dir, f'{year}_Q{quarter[-1]}.parquet')

        # save the data file
        group.to_pickle(pickle_filename)
        group.to_parquet(parquet_filename)

print("Data processing and saving to Pickle files complete.")

Processing chunk 1...
22       1980Q3
27       1980Q3
33       1980Q3
38       1980Q3
40       1980Q3
          ...  
99984    1984Q2
99988    1984Q2
99995    1984Q2
99997    1984Q2
99999    1984Q2
Name: quarter, Length: 55030, dtype: object
Processing chunk 2...
100000    1984Q2
100001    1984Q2
100003    1984Q2
100008    1984Q2
100009    1984Q2
           ...  
199994    1986Q3
199996    1986Q3
199997    1986Q3
199998    1986Q3
199999    1986Q3
Name: quarter, Length: 50289, dtype: object
Processing chunk 3...
200001    1986Q3
200004    1986Q3
200006    1986Q3
200007    1986Q3
200008    1986Q3
           ...  
299986    1988Q3
299987    1988Q3
299990    1988Q3
299992    1988Q3
299995    1988Q3
Name: quarter, Length: 44393, dtype: object
Processing chunk 4...
300000    1988Q3
300002    1988Q3
300004    1988Q3
300006    1988Q3
300010    1988Q3
           ...  
399975    1989Q4
399976    1989Q4
399981    1989Q4
399988    1989Q4
399999    1989Q4
Name: quarter, Length: 35903, dtype: object

In [5]:
print(pickle_filename)

../Data/dsws_Data/2021\2021_Q2.pkl


Chunking data into hDF5 files

In [ ]:
import os
import pandas as pd

# Path to your large .dta file (replace this with your actual file path)
file_path = '../Data/dsws_variables_monthly_basic_2021.dta'

# Directory to save the organized data by year and quarter
output_dir = '../Data/dsws_Data/'  # Update this
os.makedirs(output_dir, exist_ok=True)  # Create directory if it doesn't exist

# Define chunk size (e.g., 100,000 rows at a time)
chunk_size = 100000

# Function to extract the year and quarter from the data
def extract_year_quarter(df, date_column):
    df['year'] = pd.to_datetime(df[date_column]).dt.year
    df['quarter'] = pd.to_datetime(df[date_column]).dt.to_period('Q')
    
    # Convert 'quarter' to a string to avoid issues with HDF5
    df['quarter'] = df['quarter'].astype(str)
    
    # Return a new DataFrame with both columns at once to avoid fragmentation
    return df

# Read the .dta file in chunks
chunk_iter = pd.read_stata(file_path, chunksize=chunk_size)

# Process each chunk
for i, chunk in enumerate(chunk_iter):
    print(f'Processing chunk {i+1}...')
    
    # Extract year and quarter (assuming you have a date column like 'date')
    chunk = extract_year_quarter(chunk, 'date')  # Update 'date' to your actual date column

    # Group the chunk by year and quarter, and save it into separate HDF5 files
    for (year, quarter), group in chunk.groupby(['year', 'quarter']):
        # Create a subdirectory for each year
        year_dir = os.path.join(output_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)

        # Define the filename for the HDF5 file for each quarter
        hdf5_filename = os.path.join(year_dir, f'{year}_Q{quarter[-1]}.h5')

        # Define the key for the HDF5 file
        hdf_key = f'year_{year}_Q{quarter[-1]}'
        
        # Open or create the HDF5 file and append the data using 'fixed' format
        with pd.HDFStore(hdf5_filename, mode='a') as hdf_store:
            hdf_store.put(hdf_key, group, format='fixed')  # Use 'fixed' format for better performance

print("Data processing and saving to HDF5 files complete.")